## <font color='CC3D3D'> 1. 과제 설명
- 앱 사용성 데이터를 통한 대출신청 예측분석
    - 가명화된 데이터를 기반으로 고객의 대출상품 신청여부 예측 (2022년 3~5월 데이터제공 / 2022년 6월 예측)
    - 예측모델을 활용하여 탐색적 데이터 분석 수행
    - 대출신청, 미신청 고객을 분류하여 고객의 특성 분석결과 도출

## <font color='CC3D3D'> 2. 제공데이터
- 신용정보 데이터 : 참가신청 후, 마이페이지에서 다운로드 가능

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 100, 'max_rows', 100)

In [2]:
user_spec = pd.read_csv('./2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv')
loan_result = pd.read_csv('./2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/loan_result.csv')
log_data = pd.read_csv('./2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/log_data.csv')

### 2-1. user_spec (1,394,216, 17)
- user 신용정보

|컬럼ID|컬럼명|암호화 및 가명화|기타|
|:------|:------|:------|:------|
|application_id|신청서 번호|Y|
|user_id|유저 번호|Y|405213명|
|birth_year|유저 생년월일|
|gender|유저 성별||1(934321, 남자), 0(446934, 여자)|
|insert_time|생성일시|
|credit_score|한도조회 당시 유저 신용점수|10점 단위 round| 60-1000, 638|
|yearly_income|연소득|100만단위 반올림| 0-100억, 4,345만원|
|income_type|근로형태||'PRIVATEBUSINESS(개인사업자)', 'FREELANCER(프리랜서)', 'EARNEDINCOME(4대 보험O 직장가입자)', 'OTHERINCOME(기타소득)', 'EARNEDINCOME2(4대 보험X 직장가입자)', 'PRACTITIONER(전문직)'|
|company_enter_month|입사연월|
|employment_type|고용형태||'기타', '정규직', '계약직', '일용직'|
|houseown_type|주거소유형태||'자가', '기타가족소유', '전월세', '배우자'|
|desired_amount|대출희망금액|100만단위 반올림| 0-100억, 3,040만원|
|purpose|대출 목적||'기타(ETC)', '대환대출(SWITCHLOAN)', '생활비(LIVING)', '사업자금(BUSINESS)', '주택구입(BUYHOUSE)', '전월세보증금(HOUSEDEPOSIT)', '투자(INVEST)', '자동차구입(BUYCAR)'|
|personal_rehabilitation_yn|개인회생자 여부||0(794046, 개인회생자X), 1(12709, 개인회생자O)|
|personal_rehabilitation_complete_yn|개인회생자 납입 완료 여부||0(189510, 개인회생자인 경우 납입중), 1(1352, 납입완료)|
|national_health_insurance_type|건강보험 타입||없음|
|existing_loan_cnt|기대출수||1-278, 4.74|
|existing_loan_amt|기대출금액||0-75억, 7,074만원|

In [3]:
user_spec.user_id.nunique()

405213

In [3]:
user_spec.head()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0


### 2-2. loan_result (13,527,363, 7)
- 사용자가 선정한 대출별 금융사별 승인결과

|컬럼ID|컬럼명|암호화 및 가명화|기타|
|:------|:------|:------|:------|
|application_id|신청서 번호|Y|
|loanapply_insert_time|한도조회 일시| |2022년 3-6월|
|bank_id|금융사 번호|Y|1-64, (53제외)|
|product_id|상품 번호|Y|188가지|
|loan_limit|승인한도|십만단위 반올림|0-2,308억원, 2,274만원|
|loan_rate|승인금리|둘째자리에서 반올림|1.5-20, 12.75|
|is_applied|신청 여부(타켓)|예측레이블|0(9,709,675), 1(560,449), NAN(4,378,137)|

In [4]:
loan_result.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN


### 2-3. log_data (17,843,993, 6)
- finda App 로그 정보

|컬럼ID|컬럼명|암호화 및 가명화|기타|
|:------|:------|:------|:------|
|user_id|유저 번호|Y||
|event|행동명|||
|timestamp|행동일시||2022년 3-6월|
|mp_os|OS 기종||설명서 없음, 'Android(12331685)', 'android(2195434)', 'iOS(2654184)', 'iPadOS(2093)'|
|mp_app_version|OS 버전||설명서 없음, 'Android(90)', 'android(111)', 'iOS(93)', 'iPadOS(11)'|
|date_cd|일 코드||2022년 3-6월|

- event

|event|event_new|event description|
|:------|:------|:------|
|Clicked Signup Completion Button|SignUp|회원가입|
|Opened Finda App|OpenApp|핀다 앱 실행|
|Login|Login|핀다 앱 로그인|
|Viewed LA Intro|ViewLoanApplyIntro|한도조회 인트로 페이지 조회|
|MY LA Click Loan Apply Start 1|StartLoanApply|한도조회 시작하기 버튼 클릭|
|Completed LA ID Certification/Completed ID Certification|CompleteIDCertification|본인인증완료|
|Viewed LA Home/Viewed LA Home No Result|EndLoanApply|한도조회 결과 확인|
|LM_home_view(신)/LM Main View All 1(구)|UseLoanManage|대출관리 서비스 이용|
|Clicked Prepay Calc Btn|UsePrepayCalc|여윳돈 계산기 서비스 이용|
|Clicked Debt Calc Btn|UseDSRCalc|DSR 계산기 서비스 이용|
|대출관리_신용리포트_생성, 대출관리_신용리포트_생성_v2|GetCreditInfo|KCB 신용정보 조회(?)|

In [5]:
log_data.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25


---